### Setting Up Conda Environment with Python 3.12 and Installing Required Libraries
##### Step 1 - Created a Conda Environment with Python 3.12 and Connected to its Kernel
I created a new Conda environment with Python 3.12 and activated it. After that, I connected to the Python kernel in Jupyter/VSCode using this environment.

Commands to create and activate the environment:
```bash
conda create -n llm_env python=3.12
conda activate llm_env
```

##### Step 2 - Imported Required Libraries
Once the environment was set up and activated, I imported the following libraries for my project:
- `numpy` as `np`: For numerical operations and working with arrays.
- `pandas` as `pd`: For data manipulation and analysis.
- `torch`: PyTorch, for deep learning model building and training.
- `tqdm.notebook`: For displaying progress bars in Jupyter notebooks.
- `sklearn.preprocessing.LabelEncoder`: For encoding categorical labels into numerical format.
- `sklearn.model_selection.train_test_split`: For splitting datasets into training and testing sets.
- `transformers.BertTokenizer`: For tokenizing text data for BERT models.
- `transformers.BertForSequenceClassification`: For using BERT for sequence classification tasks.
- `torch.utils.data.TensorDataset`: For creating datasets from tensors.
- `torch.utils.data.DataLoader`: For batching datasets and loading them efficiently during training.
- `torch.utils.data.RandomSampler` and `torch.utils.data.SequentialSampler`: For random and sequential sampling of datasets.
- `transformers.AdamW` and `transformers.get_linear_schedule_with_warmup`: For the AdamW optimizer and learning rate scheduler used with transformers.
- `sklearn.metrics.f1_score`: For evaluating model performance using the F1 score.
- `random`: For random number generation, typically used in data shuffling.

Commands to install the libraries:
```bash
conda install numpy pandas scikit-learn tqdm pytorch
conda install -c huggingface transformers
```

Once the libraries were installed, I successfully imported them into the notebook and was ready to proceed with building and training the machine learning model.

In [270]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
# import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [271]:
num_epochs = 4
learning_rate = 1e-5

### Data Load and Handling

- **`df`**: The full dataset loaded from the CSV (`dataset.csv`).
- **`labels`**: The list of labels (i.e., 'Artifact Id') extracted from the dataset.
- **`label_counts`**: The count of occurrences of each unique label in the dataset.
- **`filtered_labels`**: The labels that appear at least 5 times in the dataset.
- **`filtered_labels_list`**: A list of labels that appear at least 5 times.
- **`filtered_df`**: The filtered DataFrame containing only the rows with labels that appear at least 5 times.

By running this script, you get a new DataFrame, `filtered_df`, containing only the rows with labels that appear 5 or more times in the original dataset.

**Note**: This is a very simple preprocessing step where we filter out labels with fewer than 5 occurrences. There is no further data cleaning, such as handling missing values or data normalization, applied at this stage.

In [272]:
# df = pd.read_csv('dataset.csv')

# labels = df['Artifact Id']

# label_counts = labels.value_counts()

# filtered_labels = label_counts[label_counts >= 5]

# filtered_labels_list = filtered_labels.index.tolist()

# filtered_df = df[df['Artifact Id'].isin(filtered_labels_list)]

# print(filtered_df['Artifact Id'].value_counts())

### Testing for Overfitting on a Small Portion of the Data

In this part of the code, we filter the dataset to only include labels that have exactly **16 samples**. By training the model on this small subset, we aim to observe if it achieves **overfitting**, where it memorizes the data rather than generalizing well. This helps us test the model's ability to avoid overfitting on limited data.


In [273]:
# df = pd.read_csv('dataset.csv')

# labels = df['Artifact Id']

# label_counts = labels.value_counts()

# filtered_labels = label_counts[label_counts == 16]

# filtered_labels_list = filtered_labels.index.tolist()

# filtered_df = df[df['Artifact Id'].isin(filtered_labels_list)]

### Balancing the Dataset

To address class imbalance in the dataset, we performed the following steps:

1. **Random Sampling of 'Command' Class**: We noticed that the **'d3f:Command'** class was underrepresented in the dataset. Therefore, we **randomly sampled 16 instances** from this class to ensure it is adequately represented.

2. **Combining the Data**: After sampling the 'Command' class, we combined it with the filtered dataset to create a **more balanced dataset**.

This process ensures that the model is not biased towards the larger classes and helps it generalize better across all classes.


In [274]:
df = pd.read_csv('dataset.csv')

labels = df['Artifact Id']

label_counts = labels.value_counts()

filtered_labels = label_counts[(label_counts >= 5) & (label_counts <= 200)] # Remove "Command" label

filtered_labels_list = filtered_labels.index.tolist()

filtered_df = df[df['Artifact Id'].isin(filtered_labels_list)]

command_df = df[df['Artifact Id'] == 'd3f:Command']
sampled_command_df = command_df.sample(n=16, random_state=42)
combined_df = pd.concat([filtered_df, sampled_command_df])

combined_df.reset_index(drop=True, inplace=True)

print(combined_df['Artifact Id'].value_counts())

Artifact Id
d3f:Database                     16
d3f:Software                     16
d3f:Command                      16
d3f:HardwareDriver               14
d3f:DisplayServer                11
d3f:OperatingSystem               8
d3f:FileSystem                    7
d3f:BootLoader                    6
d3f:InterprocessCommunication     5
Name: count, dtype: int64


### Divide to Train, Validation, and Test While Keeping the Distribution

In this step, the dataset is split into a training set, a validation set, and a testing set, while maintaining the same label distribution across all sets. This is done using the `train_test_split` function from `sklearn.model_selection`, with the `stratify` parameter set to the labels (`Artifact Id`) to ensure that all splits preserve the same class proportions as the original dataset.

- **Training Set**: Contains 64% of the data.
- **Validation Set**: Contains 16% of the data.
- **Test Set**: Contains 20% of the data.

By using stratified splitting, the label distribution in all sets (training, validation, and test) is consistent, preventing potential bias caused by imbalanced labels in smaller datasets.


In [275]:
train_df, test_df = train_test_split(filtered_df,
                                     test_size=0.2,
                                     stratify=filtered_df['Artifact Id'],
                                     random_state=42)

train_df, val_df = train_test_split(train_df,
                                    test_size=0.2,
                                    stratify=train_df['Artifact Id'],
                                    random_state=42)

In [276]:
print("Training set label distribution:")
print(train_df['Artifact Id'].value_counts())

Training set label distribution:
Artifact Id
d3f:Software                     10
d3f:Database                     10
d3f:HardwareDriver                9
d3f:DisplayServer                 7
d3f:OperatingSystem               5
d3f:BootLoader                    4
d3f:FileSystem                    4
d3f:InterprocessCommunication     3
Name: count, dtype: int64


In [277]:
# print("\nValidation set label distribution:")
# print(val_df['Artifact Id'].value_counts())

In [278]:
print("\nTest set label distribution:")
print(test_df['Artifact Id'].value_counts())


Test set label distribution:
Artifact Id
d3f:Software                     3
d3f:Database                     3
d3f:HardwareDriver               3
d3f:OperatingSystem              2
d3f:DisplayServer                2
d3f:FileSystem                   2
d3f:InterprocessCommunication    1
d3f:BootLoader                   1
Name: count, dtype: int64


### Model and Tokenizer Initialization

- **Model**: A pre-trained `BertForSequenceClassification` model is loaded from `bert-base-uncased`, with a classification head set to `num_labels` (number of unique labels in the dataset).

- **Tokenizer**: The `BertTokenizer` for `bert-base-uncased` is used for consistent tokenization.

These components enable fine-tuning of the BERT model for the multiclass classification task.


In [279]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(filtered_labels_list))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Data Preparation: Tokenization and Dataloader Creation

- **`tokenize_data`**: Tokenizes input text using the BERT tokenizer, padding and truncating to a max length of 512 tokens, and returns PyTorch tensors.

- **Tokenization**: The training, validation, and test data (`'Example Description'`) are tokenized into input IDs and attention masks.

- **Label Encoding**: Labels (`'Artifact Id'`) are mapped to indices and converted to tensor labels for all datasets.

- **Dataset Creation**: `TensorDataset` objects are created for training, validation, and test data with tokenized inputs and labels.

- **DataLoader**: Three DataLoaders are created:
  - `train_loader`: Random sampling for training.
  - `val_loader` and `test_loader`: Sequential sampling for validation and testing, with a batch size of 16.

These steps prepare the data for efficient use in training, validation, and evaluation.


In [280]:
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

# Tokenize the training, validation, and test data
train_encodings = tokenize_data(train_df['Example Description'].tolist())
val_encodings = tokenize_data(val_df['Example Description'].tolist())  # Validation set
test_encodings = tokenize_data(test_df['Example Description'].tolist())

# Convert labels to tensors
train_labels = torch.tensor(train_df['Artifact Id'].map(lambda x: filtered_labels_list.index(x)).tolist())
val_labels = torch.tensor(val_df['Artifact Id'].map(lambda x: filtered_labels_list.index(x)).tolist())  # Validation set
test_labels = torch.tensor(test_df['Artifact Id'].map(lambda x: filtered_labels_list.index(x)).tolist())

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)  # Validation set
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)  # Validation set
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=16)


### Optimizer and Learning Rate Scheduler

- **Optimizer**: 
  - `AdamW` is used, suitable for fine-tuning transformer models like BERT, with a learning rate of `5e-5`.
  
- **Learning Rate Scheduler**: 
  - A linear scheduler adjusts the learning rate over `total_steps` (batches × epochs), with no warm-up phase (`num_warmup_steps=0`).

These components ensure efficient model optimization during training.


In [281]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

total_steps = len(train_loader) * num_epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

c:\Users\tamar\anaconda3\envs\llm_env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Training Function

This function trains the model for a specified number of epochs:

1. **Forward Pass**: For each batch, the model computes the loss and logits, then makes predictions by selecting the class with the highest logit.
2. **Metrics Calculation**: It tracks the total loss, accuracy, and F1 score across all batches in each epoch. Accuracy is calculated by comparing the predicted labels to the true labels, and the F1 score is computed using the weighted average across all classes.
3. **Optimization**: The model parameters are updated using backpropagation (`loss.backward()`), and the optimizer and learning rate scheduler are updated each step.

At the end of each epoch, the loss, accuracy, and F1 score are printed.



In [282]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [283]:
def train(model, train_loader, optimizer, scheduler, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        all_labels = []
        all_preds = []

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()

            input_ids, attention_mask, labels = [item.to(device) for item in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            
            preds = torch.argmax(logits, dim=-1)

            # Store predictions and labels for F1 score calculation
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

            # Accuracy calculation
            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += labels.size(0)

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        # Calculate metrics for the epoch
        accuracy = total_correct / total_samples * 100
        f1 = f1_score(all_labels, all_preds, average="weighted")

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Accuracy: {accuracy:.2f}%, F1 Score: {f1:.4f}")

### Training with Validation

The `train_with_validation` function trains the model for multiple epochs and evaluates it on a validation set after each epoch. 

1. **Training**: For each batch, the model computes the loss, updates parameters, and tracks accuracy and F1 score.
2. **Validation**: After each epoch, the model's performance is evaluated on the validation set, and validation accuracy and F1 score are calculated.

Metrics (training and validation accuracy, F1 score) are printed after each epoch.


In [284]:
def train_with_validation(model, train_loader, val_loader, optimizer, scheduler, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        all_labels = []
        all_preds = []

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()

            input_ids, attention_mask, labels = [item.to(device) for item in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += labels.size(0)

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        # Calculate training metrics
        accuracy = total_correct / total_samples * 100
        f1 = f1_score(all_labels, all_preds, average="weighted")
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_samples = 0
        val_all_labels = []
        val_all_preds = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{num_epochs}"):
                input_ids, attention_mask, labels = [item.to(device) for item in batch]

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                preds = torch.argmax(logits, dim=-1)

                val_all_preds.extend(preds.cpu().tolist())
                val_all_labels.extend(labels.cpu().tolist())

                correct = (preds == labels).sum().item()
                val_correct += correct
                val_samples += labels.size(0)

                val_loss += loss.item()

        val_accuracy = val_correct / val_samples * 100
        val_f1 = f1_score(val_all_labels, val_all_preds, average="weighted")

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.6f}, Accuracy: {accuracy:.6f}%, F1 Score: {f1:.6f}")
        print(f"Validation Accuracy: {val_accuracy:.6f}%, Validation F1 Score: {val_f1:.6f}")


In [285]:
# train(
#     model=model,
#     train_loader=train_loader,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     device=device,
#     num_epochs=num_epochs
# )

train_with_validation(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    num_epochs=num_epochs
)

Training Epoch 1/4:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Model Evaluation

After training, the model is evaluated on the test set to assess its performance on unseen data. During this phase, the model is set to evaluation mode using `model.eval()`, and predictions are made on the test set.

- **Prediction**: For each batch in the test set, the model generates predictions (logits), which are converted to class labels using `torch.argmax()`.
  
- **F1 Score**: The F1 score, calculated using the `f1_score` function from `sklearn.metrics`, is reported to measure the weighted average performance across all classes.
  
- **Accuracy**: The accuracy of the model is calculated by comparing the predicted labels to the true labels and computing the percentage of correct predictions.

The evaluation results help in understanding the model's effectiveness and generalization to the test set.

*Note: This evaluation process does not update the model parameters, ensuring that it is a true test of performance.*


In [ ]:
def test(model, test_loader, device):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad(): 
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids, attention_mask, labels = [item.to(device) for item in batch]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate F1 score (weighted) for multiclass classification
    f1 = f1_score(true_labels, predictions, average='weighted')

    # Calculate accuracy
    accuracy = np.sum(np.array(predictions) == np.array(true_labels)) / len(true_labels) * 100

    print(f"F1 Score (Weighted): {f1:.4f}")
    print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
test(model, test_loader, device)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

F1 Score (Weighted): 0.1647
Accuracy: 29.41%
